In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import itertools

import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

print(tf.__version__)

# Make NumPy printouts easier to read.
np.set_printoptions(precision=3, suppress=True)

2.8.0


In [2]:
#trace = pd.HDFStore("test_profile_output-0.prof.h5") 
trace = pd.HDFStore("dpotrf-0.prof.h5")
data = trace.get('/events')
trace.close()
data.head()

,type,nb_data_items,stream_id,taskpool_id,total_data_size,begin,task_class_id,id,end,flags,chore_id,node_id,priority
0,3,0,0,2,0,320456,1,0,340988,1,0,0,-1
1,3,1,0,2,4,360547,0,0,376960,1,0,0,0
2,3,1,0,2,4,410859,0,1,421818,1,0,0,0
3,3,1,0,2,4,431244,0,2,438438,1,0,0,0
4,3,1,0,2,4,445088,0,3,451921,1,0,0,0


In [33]:
data_refined = data[['begin', 'end', 'taskpool_id', 'task_class_id','chore_id', 'nb_data_items', 'total_data_size', 'priority']].copy()
data_refined['exec_time'] = data['end'] - data['begin']
data_refined = data_refined[data_refined.priority > -1]
data_refined = data_refined.astype(float)
data_refined.head()

,begin,end,taskpool_id,task_class_id,chore_id,nb_data_items,total_data_size,priority,exec_time
1,360547.0,376960.0,2.0,0.0,0.0,1.0,4.0,0.0,16413.0
2,410859.0,421818.0,2.0,0.0,0.0,1.0,4.0,0.0,10959.0
3,431244.0,438438.0,2.0,0.0,0.0,1.0,4.0,0.0,7194.0
4,445088.0,451921.0,2.0,0.0,0.0,1.0,4.0,0.0,6833.0
5,458143.0,465029.0,2.0,0.0,0.0,1.0,4.0,0.0,6886.0


In [41]:
train_dataset = data_refined.sample(frac=0.8, random_state=0)
test_dataset = data_refined.drop(train_dataset.index)

In [42]:
COLUMNS = ['begin', 'end', 'taskpool_id', 'task_class_id','chore_id', 'nb_data_items', 'total_data_size', 'priority', 'exec_time']
FEATURES = ['begin', 'end', 'taskpool_id', 'task_class_id','chore_id', 'nb_data_items', 'total_data_size', 'priority']
LABEL = 'exec_time'

In [43]:
feature_cols = [tf.feature_column.numeric_column(k) for k in FEATURES]

In [36]:
estimator = tf.estimator.LinearRegressor(    
        feature_columns=feature_cols,   
        model_dir="train")

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'train', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [44]:
def get_input_fn(data_set, num_epochs=None, n_batch = 128, shuffle=True):    
         return tf.compat.v1.estimator.inputs.pandas_input_fn(       
         x=pd.DataFrame({k: data_set[k].values for k in FEATURES}),       
         y = pd.Series(data_set[LABEL].values),       
         batch_size=n_batch,          
         num_epochs=num_epochs,       
         shuffle=shuffle)

In [50]:
estimator.train(input_fn=get_input_fn(train_dataset, num_epochs=None, n_batch = 5, shuffle=False),steps=1000)

INFO:tensorflow:Calling model_fn.


/usr/local/lib/python3.9/site-packages/tensorflow_estimator/python/estimator/canned/linear.py:1468: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  self.bias = self.add_variable(


INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from train/model.ckpt-2000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 2000...
INFO:tensorflow:Saving checkpoints for 2000 into train/model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 2000...
INFO:tensorflow:loss = 4384502.5, step = 2000
INFO:tensorflow:global_step/sec: 350.102
INFO:tensorflow:loss = 39223770.0, step = 2100 (0.288 sec)
INFO:tensorflow:global_step/sec: 496.175
INFO:tensorflow:loss = 8852368.0, step = 2200 (0.202 sec)
INFO:tensorflow:global_step/sec: 421.823
INFO:tensorflow:loss = 34489340.0, step = 2300 (0.237 sec)
INFO:tensorflow:global_step/sec: 562.521
INFO:tensorflow:loss = 28002490.0, step = 2400 (0.180 sec)
INFO:tensorflow:global_step/sec: 477.726
INFO:tensorflow:loss = 12

In [51]:
ev = estimator.evaluate(input_fn=get_input_fn(test_dataset, num_epochs=1, n_batch = 128, shuffle=False))

INFO:tensorflow:Calling model_fn.


/usr/local/lib/python3.9/site-packages/tensorflow_estimator/python/estimator/canned/linear.py:1468: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  self.bias = self.add_variable(


INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2022-04-01T16:53:08
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from train/model.ckpt-3000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.43205s
INFO:tensorflow:Finished evaluation at 2022-04-01-16:53:09
INFO:tensorflow:Saving dict for global step 3000: average_loss = 2791265.2, global_step = 3000, label/mean = 8168.0, loss = 2791265.2, prediction/mean = 9838.708
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 3000: train/model.ckpt-3000


In [52]:
loss_score = ev["loss"]
print("Loss: {0:f}".format(loss_score))

Loss: 2791265.250000


In [53]:
train_dataset['exec_time'].describe()

count        9.000000
mean     12339.333333
std       6088.453827
min       6833.000000
25%       7194.000000
50%      10959.000000
75%      15128.000000
max      24457.000000
Name: exec_time, dtype: float64

In [40]:
#y = estimator.predict(input_fn=get_input_fn(prediction_set, num_epochs=1, n_batch = 128, shuffle=False))